# [NML-25] Notebook 5: Graph Neural Networks

TAs: [Sevda Öğüt](https://people.epfl.ch/sevda.ogut)

## Instructions

**Expected output:**

You will have coding and theoretical questions. Coding exercises shall be solved within the specified space:
```python
# Your solution here ###########################################################
...
#^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
```
Sometimes we provide variable names, such as `x = ...`; do not change names and stick to hinted typing, as they will be reused later.
Within the solution space, you can declare any other variable or function that you might need, but anything outside these lines shall not be changed, or it will invalidate your answers.

Theoretical questions shall be answered in the following markdown cell. The first line will be
```markdown
**Your answer here:**
...
```

**Solutions:**
* Your code should be self-contained in the `.ipynb` file. The solution to the exercices will be provided in an external `.ipynb` file.

* Try to make your code clean and readable, it is a good training for the project. Provide meaningful variable names and comment where needed.

* You cannot import any other library than we imported, unless explicitly stated.

## Objective

This assignment focuses on Graph Neural Networks.
In the first part, you will load and prepare data using the PyTorch Geometric library, define a GNN, implement train and test functions, and comment on the results.
In the second part, you will define a new GNN and include it in the previous architecture.

Please make sure you install the packages in `requirements.txt` respective to this notebook in your environment. Make sure that your Python version is not greater than 3.11!

In [1]:
# %pip install -r requirements.txt

In [2]:
import torch

Note that wheels are currently not available for M1/M2/M3 macs. Please install the extension packages (pyg_lib and torch_sparse) from source if you have mac. See the [installation documentation](https://pytorch-geometric.readthedocs.io/en/latest/install/installation.html) for details!

In [3]:
# %pip install pyg_lib torch_sparse -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html

## Prerequisites

The additional [tutorial](gdl_tutorial.ipynb) provides a broad overview of PyTorch and PyTorch Geometric, showing how to manipulate tensors and train neural networks as well as graph neural networks.

The following resources might help you familiarize with PyTorch and PyTorch geometric.

* [PyTorch: Learn the Basics](https://pytorch.org/tutorials/beginner/basics/intro.html)
* [PyTorch Geometric: Official Tutorials](https://pytorch-geometric.readthedocs.io/en/latest/get_started/colabs.html#official-examples)


## Section 0: Explore the data

In this section, we will go through the data to get a feeling of its content. We work with the [GitHub dataset](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.datasets.GitHub.html), from the [Multi-scale Attributed Node Embedding](https://arxiv.org/abs/1909.13021) paper. In this dataset, nodes represent developers on GitHub and edges are mutual follower relationships. It contains 37 300 nodes, 578 006 edges, 128-dimensional node features and 2 classes.

This dataset is readily available in PyTorch Geometric, so let's import relevant libraries and then install the dataset!

In [4]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import torch
import torch_geometric as pyg
from scipy import sparse
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader
from torch_geometric.datasets import GitHub
from torchmetrics import Metric
from torchmetrics.classification import Accuracy, BinaryF1Score, Precision, Recall

In [5]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu"

In [6]:
dataset = GitHub(".")
data = dataset._data

Now, we shall study its content. Node attributes are accessible through the `x` attribute, which is a `torch.Tensor`.

In [7]:
print("Design matrix")
n_nodes, n_feats = data.x.shape
print(f"Num. nodes: {n_nodes}; num features: {n_feats}")

Design matrix
Num. nodes: 37700; num features: 128


We see that we have 37 700 nodes, each with 128 features. The features correspond to an embedding of location, starred repositories, employer, and e-mail address information of each user. Each node comes with a binary label, which indicates whether it corresponds to a web or a machine learning developer.

In [8]:
print("Target vector")
print("First five elements:", data.y[:5])
print("Number of samples:", data.y.shape)
print("Number of nodes in class 1:", data.y.sum().item())

Target vector
First five elements: tensor([0, 0, 1, 0, 1])
Number of samples: torch.Size([37700])
Number of nodes in class 1: 9739


We see that the task is quite imbalanced, as class 1 is underrepresented. To get more meaningful interpretations, we swap classes zero and one.

In [9]:
data.y = 1 - data.y

The edges are contained in the `edge_index` attribute, which is again a tensor. Let's check its shape.

In [10]:
print("Edge index shape:", data.edge_index.shape)

Edge index shape: torch.Size([2, 578006])


Describe the content of the edge index matrix and how it relates to the adjacency matrix.

**Your answer here:**

The shape of this matrix is [2, num_edges], meaning that there is 578006 edges in the graph.

Now, we will create two random binary masks on the nodes: one for training and one for testing. We would like to have 80% of the samples in the training split, so we will uniformly pick nodes with that probability.

We use a masking strategy instead of directly splitting the data because our interpretation of the task is that we have a social network in which the training labels are accessible, while the test nodes, even though available, are unknown. This simplifies the sampling strategy, in particular for network methods, as we do not have to worry about loosing structure.

In [11]:
rng = torch.Generator().manual_seed(452)
train_mask = torch.randn(n_nodes, generator=rng) < 0.8

n_nodes_tr = train_mask.sum().item()
print(f"Training set size: {n_nodes_tr} ({n_nodes_tr / n_nodes:.2%})")
print(f"Test set size: {n_nodes - n_nodes_tr} ({1 - n_nodes_tr / n_nodes:.2%})")
print(
    f"Ratio of class 1 in training: {torch.sum(train_mask * data.y).item() / n_nodes_tr:.2%}"
)

Training set size: 29636 (78.61%)
Test set size: 8064 (21.39%)
Ratio of class 1 in training: 74.12%


Note that a graph with 37 700 nodes yields a 37 700 × 37 700 matrix, which has approximately 1 421 290 000 entries. Suppose that this graph is stored with a **dense** representation with 1 bit per entry, then this would result in ~170 MB of storage space. If 8 bits (1 byte) are used per entry, then this would give us ~1.42 GB. Since PyTorch uses 64-bit floats (8 bytes) per entry, we would get more than 10 GB for this graph. This illustrates that even though many entries may be zero (meaning no edge) in this 37 700 x 37 700 matrix, a dense representation allocates space for all possible node pairs. However, in a **sparse** representation, instead of storing all possible entries, only the existing edges (nonzero entries) are stored, which  can significantly reduce the required space (if you do the math it's ~2.2 MB in this example) 😄

## Section 1: Deep learning on graph data

This part presents a general workflow for deep learning and our recommended libraries: PyTorch and PyTorch Geometric. We will start with classical ML baselines, to get some robust results to which we can compare. Then we will introduce graph features, to see whether they can help in our task. Finally, we will work with deep learning and graph neural networks.

### Question 1.1: Baseline

In this question we define a baseline model with a classical ML method, namely a random forest, to get an idea of what performances we can expect from the following models. This model will only use node features, so it does not leverage at all the graph structure.

**1.1.1** Train a random forest classifier based on the node features. Make sure to use the provided `train_mask` for both the features and the target labels.

In [12]:
data.x

tensor([[ 1.4799, -0.6568, -0.1793,  ..., -0.2258,  2.0619,  0.1963],
        [-0.9020, -0.6568, -0.1793,  ..., -0.2420, -0.4832, -0.2165],
        [-0.3659, -0.6568, -0.1793,  ...,  2.5961,  2.0635, -0.2165],
        ...,
        [ 1.2091, -0.6568, -0.1793,  ..., -0.2274, -0.4849, -0.2165],
        [-0.9020, -0.6551, -0.1793,  ..., -0.2420, -0.4854, -0.2079],
        [-0.9020,  1.5236, -0.1793,  ..., -0.2420, -0.4854, -0.2165]])

In [13]:
# Your solution here ###########################################################

rf_classifier = RandomForestClassifier()

X_tr = data.x[train_mask]
y_tr = data.y[train_mask]

rf_classifier.fit(X_tr, y_tr)

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

RandomForestClassifier()

**1.1.2** Predict the labels of the test nodes, then print the `classification_report`.

In [14]:
# Your solution here ###########################################################
X_te = data.x[~train_mask]
y_te = data.y[~train_mask]

predictions = rf_classifier.predict(X_te)

print(classification_report(y_true=y_te, y_pred=predictions))
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

              precision    recall  f1-score   support

           0       0.79      0.63      0.70      2070
           1       0.88      0.94      0.91      5994

    accuracy                           0.86      8064
   macro avg       0.84      0.78      0.80      8064
weighted avg       0.86      0.86      0.86      8064



**1.1.3** Discuss what metric would be the most suitable for our setting.

**Your answer here:**

F1 score

### Question 1.2: Graph baseline - Laplacian eigenmaps

Now, let's implement a second benchmark, this time relying on structural properties. We would like to use eigenmaps of the Laplacian, but if you try to do it you would quickly run out of memory! (Go ahead and try if you wish 😉)

The adjacency matrix is too big to use it in computations, but it would be mainly filled with zeros. We can optimize memory and running time by using a **sparse representation** as discussed above.

**1.2.1** Compute the Laplacian matrix as a [sparse SciPy array][scipy_sparse]. Start by creating a sparse adjacency matrix form the `edge_index`, supposing that all edge weights are 1.

[scipy_sparse]: https://docs.scipy.org/doc/scipy/reference/sparse.html

In [16]:
def compute_laplacian(adjacency: sparse.spmatrix, normalize: str):
    """
    Parameters:
        adjacency (sparse.spmatrix): sparse adjacency matrix
        normalize (str): None, 'sym' or 'rw'

    Returns:
        L (sparse.spmatrix): Laplacian matrix (in sparse format)
    """
    diag = np.array(adjacency.sum(axis=1)).flatten()
    n = adjacency.shape[0]

    # Degree matrix as diagonal sparse matrix
    D = sparse.diags(diag)

    # Combinatorial Laplacian
    L = D - adjacency

    if normalize is None:
        return L

    # Avoid division by zero
    diag[diag == 0] = 1e-12

    if normalize == "sym":
        # D^(-1/2) A D^(-1/2)
        deg_inv_sqrt = 1.0 / np.sqrt(diag)
        D_inv_sqrt = sparse.diags(deg_inv_sqrt)
        L_norm = sparse.eye(n) - D_inv_sqrt @ adjacency @ D_inv_sqrt
        return L_norm

    elif normalize == "rw":
        # D^(-1) A
        deg_inv = 1.0 / diag
        D_inv = sparse.diags(deg_inv)
        L_norm = sparse.eye(n) - D_inv @ adjacency
        return L_norm

    else:
        raise ValueError(f"Unknown normalization: {normalize}")

In [22]:
# Your solution here ###########################################################

num_nodes = data.x.shape[0]
num_edges = data.edge_index.shape[1]
print(f"Num. nodes: {num_nodes}; num edges: {num_edges}")
adjacency = sparse.coo_matrix(
    (np.ones(data.edge_index.shape[1]), (data.edge_index[0], data.edge_index[1])),
    shape=(num_nodes, num_nodes)
)

laplacian = compute_laplacian(adjacency, normalize="sym")
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Num. nodes: 37700; num edges: 578006


**1.2.2** Use SciPy sparse linear algebra capabilities to compute the first 5 nontrivial eigenvectors of the Laplacian.

*Note: If this takes too long, you can change the condition to False after completing Question 1.2 to iterate more quickly over the following ones.*

In [31]:
from scipy.sparse.linalg import eigsh
# Your solution here ###########################################################

if True:  # Change to True to run cell

    eigvals, eigvecs = eigsh(laplacian, k=5)

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

else:
    eigvecs = np.random.rand(n_nodes, 5)

**1.2.3** Train and test a new random forest classifier using the eigenvector representation as features.

In [36]:
# Your solution here ###########################################################
rf_classifier = RandomForestClassifier()

X_tr = eigvecs[train_mask]
y_tr = data.y[train_mask]

rf_classifier.fit(X_tr, y_tr)

X_te = eigvecs[~train_mask]
y_te = data.y[~train_mask]

predictions = rf_classifier.predict(X_te)

print(classification_report(y_true=y_te, y_pred=predictions))
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

              precision    recall  f1-score   support

           0       0.62      0.34      0.44      2070
           1       0.80      0.93      0.86      5994

    accuracy                           0.78      8064
   macro avg       0.71      0.64      0.65      8064
weighted avg       0.76      0.78      0.75      8064



**1.2.4** Now, combine the two sets of features, i.e. the given programmers features and the Laplacian eigenmaps, into a single design matrix, then train and test another random forest classifier.

In [42]:
# Your solution here ###########################################################
X_tr = torch.concat([data.x[train_mask], torch.tensor(eigvecs[train_mask])], dim=1)
y_tr = data.y[train_mask]
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_tr, y_tr)
X_te = torch.concat([data.x[~train_mask], torch.tensor(eigvecs[~train_mask])], dim=1)
y_te = data.y[~train_mask]
predictions = rf_classifier.predict(X_te)
print(classification_report(y_true=y_te, y_pred=predictions))

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

              precision    recall  f1-score   support

           0       0.80      0.62      0.70      2070
           1       0.88      0.95      0.91      5994

    accuracy                           0.86      8064
   macro avg       0.84      0.78      0.80      8064
weighted avg       0.86      0.86      0.86      8064



**1.2.5** Comment on the results and describe which model you expect to perform the best on unseen data.

**Your answer here:**
...

### Question 1.3: Neural network baseline - MLP

In this question, we move from classical ML to deep learning and, again, we start from a simple model to get a viable benchmark.

**1.3.1** Create two `DataLoaders`, for the training and test data respectively, by using the `TensorDataset` class. Use the predefined batch size and shuffle training data.

References:
- [Datasets and DataLoaders](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)
- [PyTorch data utility](https://pytorch.org/docs/stable/data.html)

In [ ]:
batch_size = 128

# Your solution here ###########################################################

loader_train = ...

loader_test = ...

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

**1.3.2** Define a [module][nn_module] for a two-layer perceptron with the ReLU activation function. The hidden dimension will be a parameter of the constructor function.
This neural network will take as input a design matrix and predict the "logits" of class 1.

[nn_module]: https://pytorch.org/docs/stable/generated/torch.nn.Module.html

In [ ]:
class MLP(nn.Module):
    # Your solution here #######################################################
    def __init__(self, in_features: int, hidden_features: int): ...

    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

**1.3.3** Define a function to perform one training step for a given NN, taking as argument a batch of data `x` with target `y`, an optimizer, and a loss function. The function should return the loss value, as a float.

In [ ]:
def train_nn_step(
    optimizer: optim.Optimizer,
    loss_fn: nn.Module,
    model: nn.Module,
    x: torch.Tensor,
    y: torch.Tensor,
) -> float:
    model.train()  # Used to ensure that relevant blocks are in training mode

    # Your solution here #######################################################
    ...
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

**1.3.4** Write an evaluation function that takes as input a PyTorch Module, a data loader, and a [TorchMetrics function][torchmetrics] and returns the cumulative metric over all batches.

[TorchMetrics][torchmetrics] is a convenient package that implements metrics that work with PyTorch Tensors, and also with batched data.

[torchmetrics]: https://lightning.ai/docs/torchmetrics/stable/pages/quickstart.html

In [ ]:
def eval_nn(model: nn.Module, loader: DataLoader, metric_fn: Metric) -> float:
    model.eval()  # Used to ensure that relevant block are in evaluation model

    # Your solution here #######################################################
    ...
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

**1.3.5** Create an instance of the previously defined MLP, a relevant [loss function][torch_loss] for our classification task, and an optimizer.
When needed, as for the MLP hidden dimension and optimizer learning rate, select parameters that provide good results for the task. You might need some trial-and-error, so keep track of you results as you will be asked to comment on those hyperparameters.

Make sure to send everything to the correct device at initialization, as moving information from the CPU to the GPU is time-consuming.

[torch_loss]: https://pytorch.org/docs/stable/nn.html#loss-functions
[torch_optim]: https://pytorch.org/docs/stable/optim.html

In [ ]:
print(f"Using {device} device")

# Your solution here ###########################################################

mlp = ...

loss_fn = ...

optimizer = ...

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

**1.3.6** Perform 10 epochs of training. During each epoch, you should perform training steps iterating over the whole dataset. Gather the losses of each batch, and plot the evolution of the training loss at the end.

In [ ]:
n_epochs = 15

# Your solution here ###########################################################

...

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
plt.show()

**1.3.7** Evaluate the trained model on both the training and test data, using the most relevant metric from those already imported from TorchMetrics.

In [ ]:
# Your solution here ###########################################################

metric_fn = ...

metric_tr = ...
metric_te = ...

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

print(f"Training metric: {metric_tr:.3f}")
print(f"Test metric:     {metric_te:.3f}")

**1.3.8** Try different hyperparameters' combinations, in particular for the hidden dimension of the MLP and the learning rate of the optimizer. Then discuss the obtained results and the learning curves.

**Your answer here:**
...

### Question 1.4: Graph Neural Networks

We will now shift from the standard deep learning paradigm to Graph Neural Networks, to leverage the additional structure of our data.

We already imported [PyTorch Geometric][torch_geometric] as `pyg`, so you can access its submodules as `pyg.nn`, `pyg.data`, and so on.

[torch_geometric]: https://pytorch-geometric.readthedocs.io/en/latest/index.html

**1.4.1** Let's start by defining our first GNN. Again, it will be a subclass of the PyTorch `Module`, but this time it will take into account the `edge_index` in its `forward method`. Use two [GCN layers][gcn] to go from input features, here called *channels*, to a hidden dimension defined in the constructor, then to logit readout. Use ReLU activations.

This GNN will map node vectors to node logits.

[gcn]: https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.GCNConv.html

In [ ]:
class GCN(nn.Module):
    # Your solution here #######################################################
    def __init__(self, in_channels: int, hidden_channels: int): ...

    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

**1.4.2** Perform `n_epochs` of training of a GCN model with 64 hidden channels, using full training data as a batch. Make sure to only use training data in the loss computation by using the `train_mask`. Track the loss value at each step and plot it. Finally, evaluate the model on train and test, using the `metric_fn` from before.

In [ ]:
# Your solution here ###########################################################

gcn = ...

metric_tr, metric_te = ..., ...

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
plt.show()

print(f"Training metric: {metric_tr:.3f}")
print(f"Test metric:     {metric_te:.3f}")

**1.4.3** Hopefully, we got already some good results, but we would like to test whether stochastic optimization might be better. Batching graph data requires a particular approach, since on top of the design matrix with node features we have to account for edge information. In our setting, we have a single graph with many nodes, and a node level task. A batching strategy consists in sampling nodes with their neighbors, then working with this smaller graphs in a batched way.

Define one [NeighborLoader][neighborloader] for the training data, which will gather neighbors for as many *iterations* as layers in your GCN. Check [mini batches](https://pytorch-geometric.readthedocs.io/en/latest/get_started/introduction.html#mini-batches) if needed.

[neighborloader]: https://pytorch-geometric.readthedocs.io/en/latest/modules/loader.html#torch_geometric.loader.NeighborLoader

In [ ]:
batch_size = 1024

# Your solution here ###########################################################

loader_graph_train = ...

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

**1.4.4** Use the previously defined `train_nn_step` to train a newly initialized GCN with the new loader. Again plot the loss evolution and evaluate the trained model on train and test data.

In [ ]:
# Your solution here ###########################################################


# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
plt.show()

**1.4.5** Predict the label probabilities of each node and evaluate.

In [ ]:
# Your solution here ###########################################################

metric_tr, metric_te = ..., ...

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

print(f"Training metric: {metric_tr:.3f}")
print(f"Test metric:     {metric_te:.3f}")

**1.4.6** Define a new GNN architecture using [graph attention layers][gat].

[gat]: https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.GATv2Conv.html#torch_geometric.nn.conv.GATv2Conv

In [ ]:
class GAT(nn.Module):
    # Your solution here #######################################################
    def __init__(self, in_channels: int, hidden_channels: int): ...

    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

**1.4.7** Train and evaluate the GAT model with both methods.

In [ ]:
print("FULL TRAINING")

# Your solution here ###########################################################

metric_tr, metric_te = ..., ...

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
plt.title("Full training")
plt.show()

print(f"Training metric: {metric_tr:.3f}")
print(f"Test metric:     {metric_te:.3f}")

print("BATCH TRAINING")

# Your solution here ###########################################################

metric_tr, metric_te = ..., ...

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

plt.title("Full training")
plt.show()

print(f"Training metric: {metric_tr:.3f}")
print(f"Test metric:     {metric_te:.3f}")

## Section 2: Learning graphs

Graph attention layers are quite interesting because they dynamically weight each neighbor based on both local and incoming information from a node. This mechanism is conceptually similar to learning an entirely new graph structure on top of the existing one.

In this part, we design a block that, from node embeddings, will produce a new graph. The idea is similar to the one in the paper [Discrete Graph Structure Learning for Forecasting Multiple Time Series](https://openreview.net/forum?id=WEHSlH5mOk), which is illustrated in the following figure.

![Graph Learning Module](graph_learning_module.png)

As we can see from the schema, we have three main components:
1. **Feature extractor**: Maps each node to a new, synthesized representation.
2. **Link predictor**: for each pair of node representations, predicts the probability that an edge links them. We gather probabilities in a *structure matrix* $\theta$.
3. **Sampling**: Samples one, or multiple, discrete graphs from the structure matrix.

In the following questions, we will break down these components.

### Question 2.1: Sampling

Sampling is the most intriguing part of our module, as it maps, randomly, continuous probabilities to discrete edges. Ideally, we would like to sample each edge with a probability $\theta$, following a Bernoulli distribution, but this would be hard to backpropagate through.

What we do instead is known as the **Gumbel Trick**.
We sample edges using a [Gumbel][gumbel] reparameterization, which allows differentiating for $\theta$ through it. With $g_{ij}^1, g_{ij}^2 \sim \operatorname{Gumbel}(0,1)$ for all $i,j$, and $s$ a temperature parameter,
$$
A_{ij} = \operatorname{sigmoid}\left(
  \frac{
    \log\left( \frac{\theta_{ij}}{1 - \theta_{ij}} \right)
    + g_{ij}^1 - g_{ij}^2
  }{s}
\right)
.
$$
By letting the temperature go to zero, we can get closer and closer to a Bernoulli distribution.

[gumbel]: https://en.wikipedia.org/wiki/Gumbel_distribution

**2.1.1** Define a function to sample a matrix of Gumbel variables of given shape, knowing that, for $p$ sampled uniformly in (0,1), then $Q(p) \sim \operatorname{Gumbel}(\mu,\beta)$
$$
  Q(p)=\mu-\beta \ln (-\ln (p))
  .
$$

In [ ]:
def sample_gumbel(shape, mu=0, beta=1):
    # Your solution here #######################################################
    ...

    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


print("Testing sample_gumbel")
print(sample_gumbel((2, 3), 0, 1))

**2.1.2** Note that $log(\frac{\theta}{1 - \theta})$ is the sigmoid function, so we can work with unnormalized edge logits instead of probabilities. Define a function to sample an adjacency matrix $A$ from edge logits using the Gumbel Trick.

In [ ]:
def sample_gumbel_trick(logits, temperature, mu=0, beta=1):
    # Your solution here #######################################################
    ...
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


print("Testing sample_gumbel_trick")
print(sample_gumbel_trick(torch.tensor([1000, 0, 0, -10]), temperature=10))
print(sample_gumbel_trick(torch.tensor([1000, 0, 0, -10]), temperature=1))
print(sample_gumbel_trick(torch.tensor([1000, 0, 0, -10]), temperature=1e-2))

### Question 2.2: Link predictor

GNNs are all about node embeddings, which by now we should know how to deal with. The missing component is therefore the **link predictor**. Naively, we could iterate through all pairs of nodes and apply a predictor layer, but it would be highly inefficient. To leverage tensor manipulation, let's start by gathering paired node representations in a matrix, so that we can predict probabilities in parallel.

**2.2.1**  Define a function that takes as input a tensor of node embeddings, and returns a tensor that concatenates embeddings pairwise. Use [triu_indices][triu_indices] to have pairs appearing only once and avoid self loops, and return the indices along with the embeddings.

[triu_indices]: https://pytorch.org/docs/stable/generated/torch.triu_indices.html

In [ ]:
def pair_embeddings(x) -> (torch.Tensor, torch.Tensor):
    # Your solution here ###########################################################
    ...
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


print("Testing pair_embeddings")
print(pair_embeddings(torch.tensor([[1.0], [2.0], [3.0]]))[0])
print(pair_embeddings(torch.tensor([[[1.0], [2.0], [3.0]], [[4.0], [5.0], [6.0]]]))[0])

**2.2.2** Define a PyTorch module that takes as input node embeddings, compute link probabilities with a two-layer perceptron on paired embeddings, then samples edges with the Gumbel trick. The output of the forward method will be a PyTorch Geometric [EdgeIndex][edge_index] of tensors representing indices and weights corresponding to positively sampled edges. You might need a `eps` threshold to avoid numerical errors.

[edge_index]: https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.EdgeIndex.html

In [ ]:
class MLPGraphLearn(nn.Module):
    # Your solution here ###########################################################
    def __init__(
        self, in_features: int, hidden_features: int, temperature: float, eps=1e-10
    ): ...

    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


print("Testing MLPGraphLearn")
mlp_gl = MLPGraphLearn(2, 5, 0.01)
mlp_gl(torch.tensor([[[1.0, 2.0], [0.5, 7.1], [-0.1, 0.3]]]))

### Question 2.3: Classifiers

Let's introduce our graph learning block into some classifiers.


**2.3.1** Define a classifier that first produces node embeddings with a linear layer with ReLU activation, which it feeds to the previously defined GL module, then it performs two graph convolutions on the original node features using the learned graph.

In [ ]:
class MLPGLClassifier(nn.Module):
    # Your solution here #######################################################
    def __init__(
        self,
        in_features: int,
        gl_node_features_in: int = 64,
        gl_node_features_hidden: int = 32,
        gcn_hidden: int = 64,
    ): ...

    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

**2.3.2** Unfortunately, training and evaluating the `MLPGLClassifier` might take too long. Let's just test whether it works: instantiate the classifier and compute the graph embedding.

In [ ]:
# Your solution here ###########################################################

# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^